# **Abstract**

기존 sequence transduction model들은 인코더와 디코더를 포함한 복잡한 recurrent나 CNN에 기반

- <b>"Transformer"</b>: 온전히 attention mechanism에만 기반한 구조

<br>

---



# **Attention mechanism**

<img src = "https://user-images.githubusercontent.com/42150335/147276107-8d3b8047-82be-42be-aebd-a6f1690cf050.png">

- 딥러닝, 특히 자연어처리와 컴퓨터 비전에서 사용되는 핵심 기술

- Attention: 모델이 입력 데이터의 어떤 부분에 "주의"를 기울일지 결정하는 매커니즘
  - 인간이 긴 문장을 읽을 때 중요한 단어들에 더 집중하는 것과 비슷한 원리

- 작동 원리
  1. **Query, Key, Value**: 입력을 세 가지 벡터로 변환
    - Query: 무엇을 찾고 있는지(Target hidden state in the decoder)
    - Key: 각 위치의 식별자(All possible hidden state in the encoder)
    - Value: 실제 정보 내용(Key)
  2. **Attention Score 계산**: Query와 Key의 유사도를 계산하여 각 위치의 중요도 결정
    - score 값이 높을수록 단어가 비슷, 낮을수록 상이
<img src = "https://user-images.githubusercontent.com/42150335/147276113-12444d0c-c63f-486d-a1de-c1bbea208a5b.png">
  3. **가중합**: Score에 따라 Value들을 가중평균하여 최종 출력 생성

- 주요 장점
  - **장거리 의존성 해결**: RNN과 달리 멀리 떨어진 정보도 직접 참조 가능
  - **병렬 처리**: 순차적 처리가 아닌 병렬 계산으로 속도 향상
  - **해석 가능성**: 어떤 부분에 주의를 기울였는지 시각화 가능

# **Introduction**

Sequence Modeling과 transduction 문제에서 RNN, long-term memory, gated RNN이 SOTA(State-of-the-art, 현재 최고 수준의 결과)를 달성해 옴

> <b>Recurrent model(RNN)</b>은 parallelization(병렬화, 여러 연산을 동시에 수행)이 불가능해 longer sequence length에서 치명적

- RNN은 각 단계가 이전 단계의 결과에 의존하기 때문에 병렬화 불가
- 최근 연구에서 factorization trick과 conditional computation을 통해 계산 효율성을 많이 개선
- 특히 conditional computation은 모델 성능도 동시에 개선


***-> 그러나 여전히 근본적인 sequential computation 문제 존재***
- sequential computation 문제: 입력 시퀀스를 순차적으로 처리해야만 하는 구조적인 한계
- RNN, LSTM, GRU와 같은 순환 신경망에서 두드러지는 문제
  - 시간 순서대로 처리해야 함
  - 병렬화가 어려움
  - Long sequence일수록 느림(시간이 선형적으로 늘어남)


> <b>Attention mechanism</b>은 다양한 분야의 sequence modeling과 transduction model에서 주요하게 다뤄짐

- Attention mechnism은 input과 output sequence간 길이를 신경쓰지 않아도 됨

**<i>-> 하지만 여전히 recurrent network와 함께 사용되었음</i>**






> **기존 모델들의 문제점**

1. **RNN/LSTM의 고정 크기 문제**
  - 긴 시퀀스를 고정된 크기의 hidden state 하나로 압축
  - 문장을 하나의 벡터로 압축하며 정보 손실 불가피(bottleneck 문제) 발생

2. **Seq2Seq의 한계**
  - Encoder가 전체 입력을 하나의 context vector로 요약
  - 입력이 길수록 초반 정보 손실

> **Attention이 해결하는 방식**

1. **직접 접근(Direct Access**)<br>
  입력: [단어1, 단어2, 단어3, ..., 단어N]<br>출력 생성 시: 모든 입력 단어에 직접 접근 가능<br>
  - 거리에 관계없이 모든 위치 간 직접 연결
  - 정보 손실 없이 즉시 접근
2. **동적 가중치**
  - 출력의 각 위치에서 입력의 모든 위치를 확인
  - 관련성에 따라 가중치를 동적으로 계산
  - 필요한 정보만 선택적으로 활용

***-> Attention은 정보 손실 없이 임의 길이의 입력과 출력을 처리할 수 있도록 해줌***


> **Transformer**

- input와 output간 global dependency를 뽑아내기 위해 recurrence를 사용하지 않고, attention mechanism만을 사용
  - Global Dependency: 문장에서 멀리 떨어진 단어들 간의 의존 관계
  - recurrence
    - 순차적 정보 전달 중 각 단계마다 정보 손실의 가능성 존재, 길어질수록 초반 정보가 희미해짐(vanishing gradient)
    - 병목 현상: 모든 정보가 hidden state라는 고정 크기 벡터 통과, 긴 시퀀스에서 정보 압축으로 인한 손실

**<i>-> parallelization이 가능해 적은 시간으로 translation quality에서 SOTA를 달성할 수 있었음</i>**

<br><br>

---

# **Background**

> **sequential computation(순차 계산)을 줄이는 것**은 Extended Neural GPU, ByteNet, ConvS2S에서도 다뤄짐

- 이 연구들은 모두 CNN을 basic building block으로 사용
- input output 거리에서 dependency를 학습하기 어려움
  - CNN의 제한적 수용 범위(Receptive Field)
    - 3x3 필터: 한 번에 3개 단어만 볼 수 있음
    - 거리가 있는 단어를 연결하려면 여러 층 필요 -> 층이 깊어질수록 정보 손실 발생

<b><i>-> Tranformer에서 Multi-Head Attention을 통해 상수 시간으로 줄어듦</b></i>

> **Multi-Head Attention**

- 모든 위치 간 직접 연결
- 병렬 처리 가능
- 거리에 관계없이 동일한 계산 복잡도

<i> attention은 가중치 합 but 내적을 개별적으로 계산하기 때문에 attention 하나만으로는 위치나 순서 정보를 알 수 없음</i>

> Positional Encoding

- Transformer는 RNN과 달리 입력 시퀀스를 순차적으로 처리하지 않고 병렬적으로 처리, 때문에 입력 토큰 간의 순서를 인식할 수 없음
- 입력 단어 임베딩에 위치 정보를 더해줌

> **Self-attention( = intra-attention)**

- 시퀀스 내의 각 요소가 다른 요소들과의 관계를 고려하여 자신을 재표현하는 방법
- reading comprehension(독해)
  - 질문과 지문의 관련 부분을 정확히 연결
- abstractive summarization(추상적 요약)
  - 문서 전체를 보고 핵심 내용 추출
  - 멀리 떨어진 문장들 간의 관계 파악
- textual entailment(텍스트 함의)
  - 두 문장 간의 논리적 관계 판단
  - 전체 맥락을 고려한 추록
- learning task, independent sentence representations를 포함한 다양한 task에서 성공적으로 사용

> **End-to-end memory network**

- Recurrent Attention 사용
- 메모리에서 관련 정보를 반복적으로 검색
- Question Answering에서 좋은 성능
- sequence-aligned recurrence 보다 recurrent attention mechanism에 기반
- simple-language question answering 과 language modeling task에서 좋은 성능을 보임


> **Transformer는 온전히 self-attention에만 의존한 최초의 transduction model ( sequence-aligned RNN이나 Convolution을 사용하지 않음)**

- RNN의 순차 처리 완전 제거
- CNN의 지역적 처리 완전 제거
- 오직 Self-Attention으로만 구성
- 완전한 병렬 처리 가능
- 장거리 의존성을 효과적으로 포착
- 훨씬 빠른 학습과 추론

<br><br>

---



# **Model Architecture**

<h3><u>(1) Encoder and Decoder Stacks</u></h3>
<img src = "https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FkMerW%2FbtrCyXNESwS%2FVYFC5WtvFchhtt8SmTjb90%2Fimg.png">

transformer의 구조

> Encoder

- 번역할 대상을 받기 때문에 전체 정보를 다 봄
- Encoder는 6개의 identical layer로 이루어짐
- 각 layer는 두 개의 sub-layer를 가짐
- 첫 번째 sub-layer는 multi-head self-attention mechanism
- 두 번째 sub-layer는 간단한 position-wize fully connected feed-forward network
- 각 two sub-layers 마다 layer nomalization 후에 residual connection을 사용
- 즉 각 sub-layer의 결과는 <i>LayerNorm(x + Sublayer(x))</i>
- residual connection을 구현하기 위해, embedding layer를 포함한 모든 sub-layer들의 output은 512차원
  - d<sub>model</sub> = 512

> Decoder

- 번역할 문장을 보고 tartget 언어의 단어 순서대로 문장 생성
- Decoder도 마찬가지로 6개의 identical layer로 이루어짐
- 각 Encoder layer의 두 sub-layer에, decoder는 세 번째 sub-layer를 추가
  - encoder stack의 결과에 해당 layer가 multi-head attention을 수행
- 마찬가지로 residual connection 적용
- masking: 예측할 단어 뒤의 미래 단어들이 사용되지 않도록 내적 결과에 극단적인 음수 값을 주는 방식
 <br><br>





<h3><u>(2) Attention</u></h3>

Attention function은 쿼리와 key-value 쌍을 output에 매핑함(query,key,value,output은 모두 vector임)
output은 value들의 weighted sum으로 계산됨<br><br>


<h4>1. Scaled Dot-Product Attention</h4>
<img src = "https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2Fdduf4u%2FbtrCDdgFUom%2FgKFu7qrcC21ToxmIucvKEK%2Fimg.png">

- input: query,  key의 dimension d<sub>k</sub>, value의 dimension d<sub>y</sub>
- 모든 쿼리와 key에 대해 dot product를 계산하고, &radic;d<sub>k</sub>로 나눠주고, weight를 적용하기 위해 value에 softmax 함수를 적용
- <img src="https://i.ibb.co/bMyS58v8/2025-06-01-011122.png">

두 가지 Attention function이 존재
1. Additive attention: single hidden layer로 feed-forward later network 사용해 compatibility funciton 계산
2. Dot-product attention: scaling factor를 제외하면 이 연구에서의 attention 방식과 동일

d<sub>k</sub>가 작으면 두 방식의 성능은 비슷하지만, d<sub>k</sub>가 큰 경우 additive가 더 성능이 좋음
d<sub>k</sub>가 크면 dot product의 경우 gradient가 너무 작아지는 문제를 해결하기 위해 dot product를 1/d<sub>k</sub>로 스케일링함<br><br>

<h4>2. Multi-Head Attention</h4>
<img src = "https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2Fbr2h4n%2FbtrCHq8u8zV%2F80K0Ej32EMvbJUmYpgZ2hK%2Fimg.png">

Single attention을 d<sub>model</sub>-dimensional keys, values, queries에 적용하는 것보다, queries, keys, values를 h번 서로 다른, 학습된 linear projection으로 d<sub>k</sub>, d<sub>k</sub>와 d<sub>y</sub> 차원에 linear하게 project하는 게 더 효과적이라는 사실을 알아냄

-> project된 각 값들은 병렬적으로 attention function을 거쳐 d<sub>y</sub>-dimensional output value를 만들어 냄
-> 이 결과들은 다시 합쳐진 다음, 다시 한 번 project 되어 최종 결과값을 만듦

<img src="https://i.ibb.co/XkdC4NJs/2025-06-01-011122.png">

이 연구에선 <i>h = 8이고, d<sub>k</sub> = d<sub>y</sub> = d<sub>model</sub>/h = 64</i>

-> 각 head마다 차원을 줄이기 때문에, <b>전체 계산 비용은 전체 차원에 대한 single-head attention과 비슷함 </b><br><br>

<h4>3. Applications of Attention in our Model</h4>

Transformer는 세 가지 방법으로 multi-head attention을 사용
1. 인코더-디코더 attention layers 에서
- query는 이전 디코더 layer에서 나옴
- memory key와 value는 인코더의 output에서 나옴
- -> 따라서 디코더의 모든 position이 input sequence의 모든 position을 다룸
- 전형적인 seq2seq model에서의 인코더-디코더 attention 방식

2. 인코더는 self-attention layer를 포함
- self-attention later에서 key, value, query는 모두 같은 곳(인코더의 이전 layer의 output)에서 나옴
- 인코더의 각 position은 인코더의 이전 layer의 모든 position을 다룰 수 있음

3. 디코더 또한 self-attention layer를 가짐
- 마찬가지로, 디코더의 각 position은 해당 position까지 모든 position을 다룰 수 있음
- 디코더의 leftforqard information flow는 auto-regressive property 때문에 막아줘야 할 필요가 있음
- -> 이 연구에서는 scaled-dor product attention에서 모든 softmax의 input value 중 illegal connection에 해당하는 값을 −∞로 masking out해서 구현

<h3><u>(3) Position-wize Feed-Forward Networks</u></h3>

인코더 디코더의 각 layer는 fully connected feed-forward network를 가짐
- 이는 각 position에 따로따로, 동일하게 적용
- ReLu 활성화 함수를 포함한 두 개의 선형 변환 포함

<i>FFN(x) = max(0, <sub>x</sub>W<sub>1</sub> + b<sub>1</sub>)W<sub>2</sub> + b<sub>2</sub></i>

linear transformation은 다른 position에 대해 동일하지만 layer 간 parameter는 상이

<h3><u>(4) Embeddings and Softmax</u></h3>

다른 sequence transduction models처럼, 학습된 임베딩 사용
- input 토큰과 output토큰을 d<sub>model</sub>의 벡터로 변환하기 위함

decoder ouput을 예측된 다음 토큰의 확류롤 변환하기 위해 선형 변환과 softmax 사용
- transformer에서는, 두 개의 임베딩 layer와 pre-softmax 선형 변환 간 같은 weight의 matrix를 공유

임베딩 layer에서는 weight들에 &radic;d<sub>model</sub>를 곱해줌

<h3><u>(5) Positional Encoding</u></h3>

Transformer는 어떤 recurrence, convolution도 사용하지 않기 때문에, sequence의 순서를 사용하기 위해 sequence의 상대적, 절대적 position에 대한 정보 주입 필요

인코더와 디코더 stack 아래의 input 임베딩에 **"Positional Encoding"**을 추가
- Positional Encoding은 input 임베딩처럼, 같은 차원 (d<sub>model</sub>)을 가져서, 둘을 더할 수 있음
- 다양한 positional encoding 방법 중에, transformer는 다른 주기의 sine, cosine function을 사용

<img src="https://i.ibb.co/bMGR7gYv/2025-06-01-171913.png">
- <i>pos</i>: position
- <i>i</i>: dimension
- 즉 positional encoding의 각 차원은 sine 곡선에 해당
- 모델이 상대적인 position으로 쉽게 배울 수 있을 거라 가정하여 위 functon 사용
  - 어떤 고정된 offset <i>k</i>라고 <i>PE<sub>pos+k</sub></i>가 <i>PE<sub>pos+k</sub></i>로 표현될 수 있기 때문

학습된 Positional Embedding을 사용해 실험 진행
- 두 방식은 거의 같은 결과 시사
- transformer에선 sine 곡선의 방식 선택
  - model이 더 긴 sequence 길기를 추론할 수 있게 해줌<br><br>

---




# **Why Self-Attention**

recurrent, convolution layer와 self-attention을 비교
1. layer당 전체 계산 복잡도
2. sequential parallelize 할 수 있는 계산의 양
3. network에서 long-range dependency 사이의 path 길이
  - network에서 순회해야 하는 forward와 backward의 path 길이가 이런 dependency를 학습하는 능력에 영향을 주는 주요 요인
  - input과 output sequence에서 position의 조합 간의 path가 짧을수록, long-range dependency를 합습하기가 쉬움
  - -> input과 output position 사이의 최대 path 길이를 비교할 것

<img src = "https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FwbH8Q%2FbtrCIAjYy6w%2FePDkWqf3DrZoSojdJKJ220%2Fimg.png">

self-attention layer는 모든 position을 상수 시간만에 연결

recurrent layer의 경우 <i>O(n)</i>이 소요

계산 복잡도 면에서, self-attention layer가 <i> n < d</i>일 때 recurrent layer보다 빠름
- <i>n</i>: Sequence length, <i>d</i>: representation demensionality
- n < d인 경우가 machine translation에서의 대부분의 경우에 해당

아주 긴 sequence의 경우 계산 성능 개선을 위해 self-attention은 input seqeunce의 neightborhood size를 r로 제한할 수 있음
- 이는 maximum path의 길이를 <i>O(n/r)</i>로 증가시킬 수 음

<i>k < n</i>인 kernel width의 single convolutional layer는 input과 output의 모든 쌍을 연결하지 않음

contiguos kernel의 경우 <i>O(n/k)</i>의 stack이 필요하고 dilated convolution의 경우 <i>O(log<sub>k</sub>(n))이 필요함

Convolution layer는 일반적으로 recurrent layer보다 더 비용이 많이 듦
- Seperable Convolution의 경우 복잡도를 <i>O(knd + nd<sup>2</sup>)까지 줄일 수 있음
- <i><u>그러나 k = n의 경우, transformer와 같이 self-attention layer와 point-size feed forward layer의 조합과 복잡도가 같음</u></i>

self-attention은 더 interpretable한 모델을 만들 수 있음
- attention distribution에 대해 다룸
- 각 attention head들은 다양한 task를 잘 수행해내고, 문장의 구문적, 의미적 구조를 잘 연관시키는 성질을 보이기도 함<br><br>

---

# **Training**

<h3><u>(1) Training Data and Batching</u></h3>

English-German

- WMT 2014 English-German 데이터셋
- 4.5백만 sentence pairs
- 문장들은 byte-pair 인코딩으로 인코딩 되어있음

English-French

- WMT 2014 English-French 데이터셋
- 36M sentences 와 32000 word-piece vocabulary로 쪼개진 토큰들

<h3><u>(2) HardWare and Schedule</u></h3>

- 8개의 NVIDIA P100 GPU로 학습
- base model은 12시간 동안 (100,000 step) 학습시킴
- big model 은 3.5일 동안 (300,000 step) 학습시킴

<h3><u>(3) Optimizer</u></h3>

- Adam optimizer 사용
- <img src="https://i.ibb.co/R4b7LMfG/2025-06-01-174912.png">

<h3><u>(4) Regularization</u></h3>

세 가지 regularization을 사용

residual Dropout
1. 각 sub-layer의 output에 dropout을 적용
2. 임베딩의 합과 positional 인코딩에 dropout 적용

Label Smoothing
3. 학습 중에 label smmothing 적용 (ϵ<sub>ls</sub> = 0.1)<br><br>

---

# **Results**

<h3><u>(1) Machine translation</u></h3>

WMT 2014 English-to-German translation, English-to-French translation에서 SOTA 달성

<h3><u>(2) Model Variation</u></h3>

<h3><u>(3) English Constituency Parsing</u></h3>

English Constituency Parsing에서도 잘 일반화해서 사용할 수 있는지 실험

구체적인 tuning 없이도 놀라운 성능을 보임